In [33]:
# !pip install -q textract==1.6.2

ERROR: Could not find a version that satisfies the requirement textract==1.6.2 (from versions: 0.1.0, 0.2.0, 0.3.0, 0.4.0, 0.5.0, 0.5.1, 1.0.0, 1.1.0, 1.2.0, 1.3.0, 1.4.0, 1.5.0, 1.6.0, 1.6.1, 1.6.3, 1.6.4, 1.6.5)
ERROR: No matching distribution found for textract==1.6.2


# Grammar Error Corection

In [17]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
# model_name = 'deep-learning-analytics/GrammarCorrector'
model_name = 't5_gec_model_02' # model path
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def correct_grammar(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='max_length',max_length=64, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=64,num_beams=4, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [18]:
input_text = "summary: I am enjoys, writtings Articles ons AI and I also enjoyed write articling on AI."
num_return_sequences = 3 # Số lượng câu generate
corrected_text = correct_grammar(input_text, num_return_sequences)
corrected_text

['summary: I enjoy writing articles on AI and I also enjoy writing articles on AI.',
 'Summary: I enjoy writing articles on AI and I also enjoy writing articles on AI.',
 'summary: I enjoy writing articles on AI and I also enjoyed writing articles on AI.']

In [19]:
corrected_text

['summary: I enjoy writing articles on AI and I also enjoy writing articles on AI.',
 'Summary: I enjoy writing articles on AI and I also enjoy writing articles on AI.',
 'summary: I enjoy writing articles on AI and I also enjoyed writing articles on AI.']

# Question Generation and Fact - Check

In [20]:
# import fitz  # PyMuPDF
from transformers import T5ForConditionalGeneration, T5Tokenizer
import json

model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [21]:
import re
import textract

# Đọc nội dung của tài liệu PDF
text = textract.process('pdf/Report.pdf', encoding='utf-8')

# Sử dụng biểu thức chính quy để cắt thành các đoạn văn
all_paragraphs = re.split(r'\s{2,}', text.decode('utf-8'))
num_paragraph= len(text)
print(num_paragraph)

13876


In [22]:
list_para = []
list_para = [para for para in all_paragraphs if len(para.split()) >= 20] # list of paragraphs which have more than 20 words
len(list_para)

21

In [23]:
# Summarize paragraph using t5 model

summaries = []
for i,paragraph in enumerate(list_para):
    input_text = "summarize: " + paragraph
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(input_ids, max_length=150, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    summaries.append(summary)
    
    # if i==3: break

In [24]:
list_para[:4]

['Abstract. This paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters.',
 "It has always been a tough choice for the students to choose the courses in different semesters in which there is possibility to score good grades apart from the interest in the course. IIIT-Delhi offers variety of courses with mandatory courses in first 4 semesters (with exception of 2 to 3 electives) and all elective courses from fifth semester onwards. Hence, choosing the courses based on the verbal recommendation from the seniors, instructors and fellowmates becomes a hectic task. For easing this process of course recommendation for an upcoming semester, we have developed a system which deploys simple yet powerful recommendation techniques such as auto-encoders, hybrid matrix factorization and similarity based approaches. It is a GUI based system which takes an input of student's ID (which is stored in the backend d

In [25]:
# merge into result 

result = [{"paragraph": list_para[i], "summary": summaries[i]} for i in range(len(summaries))]

In [26]:
result[:5]

[{'paragraph': 'Abstract. This paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters.',
  'summary': 'this paper aims to develop a system that will help in recommendation of courses for an upcoming semester based on the performance of previous semesters. this paper aims to develop a system that will help in recommendation of courses based on the performance of previous semesters.'},
 {'paragraph': "It has always been a tough choice for the students to choose the courses in different semesters in which there is possibility to score good grades apart from the interest in the course. IIIT-Delhi offers variety of courses with mandatory courses in first 4 semesters (with exception of 2 to 3 electives) and all elective courses from fifth semester onwards. Hence, choosing the courses based on the verbal recommendation from the seniors, instructors and fellowmates becomes a hectic task. For easing this

In [85]:
# Clone this responsitory for questions generator

# !git clone https://github.com/amontgomerie/question_generator!git clone https://github.com/amontgomerie/question_generator
# !pip install -r question_generator/requirements.txt -qq
# !python run_qg.py --text_file question_generator/articles/twitter_hack.txt

C:\Users\Sysme Hue\Desktop\LMS\t5-grammar


In [35]:
# import file from another folder

import sys
# caution: path[0] is reserved for script path (or '' in REPL)
sys.path.insert(1, 'question_generator')

from questiongenerator import QuestionGenerator
qg = QuestionGenerator()


c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


0

In [36]:
text = result[2]["summary"]
q  = qg.generate(text, num_questions=3)

Generating questions...



c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1.0 and may not be 100% compatible with the current version (3.7.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Evaluating QA pairs...



In [37]:
q

[{'question': 'how many students have taken the course?',
  'answer': 'the dataset consists 739 students and 306 subjects with mapping of each student to the grades for each course the student has taken throughout the duration of their degree.'},
 {'question': 'how many students have taken the iitt?',
  'answer': 'the dataset has been acquired from the official IIIT-Delhi academics department for the students of 7 Computer Science passout batches.'},
 {'question': 'how many subjects do students have taken?',
  'answer': [{'answer': 'dataset', 'correct': False},
   {'answer': '7 Computer Science', 'correct': False},
   {'answer': '306', 'correct': True},
   {'answer': '739', 'correct': False}]}]

In [38]:
print(text)
for i in range(len(q)): print(q[i]["question"]) #  7 Computer Science passout batches.

the dataset has been acquired from the official IIIT-Delhi academics department for the students of 7 Computer Science passout batches. the dataset consists 739 students and 306 subjects with mapping of each student to the grades for each course the student has taken throughout the duration of their degree.
how many students have taken the course?
how many students have taken the iitt?
how many subjects do students have taken?


### Fact -check

In [2]:
import torch
import torch.nn as nn

from transformers import BertTokenizer, BertModel

c:\Users\Sysme Hue\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Xây dựng mô hình

class BERTClassificationModel(nn.Module):
    def __init__(self, bert_model_name, num_labels):
        super(BERTClassificationModel, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(self.bert.config.hidden_size * 2, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = torch.cat((outputs.last_hidden_state[:, 0, :], outputs.last_hidden_state[:, -1, :]), dim=1)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# This might error but thing fine

model_path = "model/bert_classification_model.pth"

device = torch.device("cpu")

# Tạo mô hình mớia
loaded_model = BERTClassificationModel('bert-base-uncased', num_labels=3)

# Nạp trạng thái từ tệp đã lưu
loaded_model.load_state_dict(torch.load(model_path))
loaded_model.to(device)

In [10]:
# Tải lại tokenizer từ đường dẫn đã lưu
loaded_tokenizer = BertTokenizer.from_pretrained("model/tokenizer")

In [13]:
max_seq_length = 128
def predict_premise_hypothesis(premise_text, hypothesis_text, model, tokenizer):
    # Chuẩn bị dữ liệu đầu vào cho mô hình
    inputs = tokenizer(premise_text, hypothesis_text, padding=True, truncation=True, max_length=max_seq_length, return_tensors="pt")
    input_ids = inputs["input_ids"].to(device)
    attention_mask = inputs["attention_mask"].to(device)

    # Dự đoán
    model.eval()
    with torch.no_grad():
        logits = model(input_ids, attention_mask)
        probabilities = torch.softmax(logits, dim=1)
        predicted_label = torch.argmax(probabilities, dim=1).item()

    return predicted_label

pre = ["entailment", "neutral", "contradiction"]
# Sử dụng hàm predict_premise_hypothesis để dự đoán
premise_text = "The likelihood is 100%"
hypothesis_text = "Maybe the probability is 100%"
predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)

print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])


premise_text : The likelihood is 100% 
 hypothesis_text : Maybe the probability is 100%
Predicted Label: neutral


In [39]:
premise_text = "The dataset had acquired for the student of 7 computer science"
hypothesis_text = result[2]["summary"]

predicted_label = predict_premise_hypothesis(premise_text, hypothesis_text, loaded_model, loaded_tokenizer)
print("premise_text :", premise_text, "\n", "hypothesis_text :", hypothesis_text)
print("Predicted Label:", pre[predicted_label])

premise_text : The dataset had acquired for the student of 7 computer science 
 hypothesis_text : the dataset has been acquired from the official IIIT-Delhi academics department for the students of 7 Computer Science passout batches. the dataset consists 739 students and 306 subjects with mapping of each student to the grades for each course the student has taken throughout the duration of their degree.
Predicted Label: entailment
